<a href="https://colab.research.google.com/github/brianMutea/music_gen/blob/main/Music_generation_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%pip install pretty_midi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.3 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=268da5d9e77f0f50f02e816c4dc9e348d2ee5b11117c645543773121ae94d83d
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [ ]:
!sudo apt install -y fluidsynth

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth2 libinstpatch-1.0-2 qsynth
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs timidity jackd musescore
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth2 libinstpatch-1.0-2 qsynth
  timgm6mb-soundfont
0 upgraded, 6 newly installed, 0 to remove and 15 not upgraded.
Need to get 126 MB of archives.
After this operation, 157 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libinstpatch-1.0-2 amd64 1.1.2-2build1 [238 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 timgm6mb-soundfont all 1.3-3 [5,420 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libfluidsynth2 amd64 2.1.1-2 [

In [ ]:
!pip install --upgrade pyfluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
import os
from music21 import converter, instrument
import fluidsynth

import tensorflow as tf
import numpy as np
import pandas as pd
from collections import Counter
import random
import IPython
from IPython import display
import music21
from music21 import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Loading data

In [ ]:
od.download("https://www.kaggle.com/datasets/imsparsh/lakh-midi-clean")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

One way...

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

files_dir_path = '/content/lakh-midi-clean/'

In [ ]:
import fnmatch

filenames = [os.path.join(dirpath, file)
    for dirpath, dirnames, files in os.walk(files_dir_path)
    for file in files if file.endswith('.mid')]

In [ ]:
len(filenames)

In [ ]:
random_file = np.random.randint(1, 1000)
sample_file = filenames[random_file]
print(sample_file)

In [ ]:
import pretty_midi
pm = pretty_midi.PrettyMIDI(sample_file)

In [ ]:
import fluidsynth
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=45):
  waveform = pm.fluidsynth(fs=30000)
  waveform_short = waveform[:seconds*30000]
  return display.Audio(waveform_short, rate=30000)

In [ ]:
display_audio(pm)

Check the instruments used in the midi files...

In [ ]:
import pretty_midi

instruments = []
for instrument in pm.instruments:
  # check the number of instruments
  # instrument = pm.instruments[0]
  instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
  instruments.append(instrument_name)
print('Number of instruments: ', len(instruments))
print('Kinds of instruments:\n', instruments)

In [ ]:
# Extracting the notes
for i, note in enumerate(instrument.notes[:10]):
      note_name = pretty_midi.note_number_to_name(note.pitch)
      duration = note.end - note.start
      print(f'{i}: pitch={note.pitch}, note_name={note_name}, duration={duration:.4f}')



Extract the notes

In [ ]:
from collections import defaultdict
def extract_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = defaultdict(list)

  sort_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sort_notes[0].start

  for note in sort_notes:
    start = note.start
    end = note.end
    # notes['pitch'].append(note.pitch)
    notes['note type'].append(pretty_midi.note_number_to_name(note.pitch))
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [ ]:
notes = extract_notes(sample_file)
notes.head()

# Create training data